In [47]:
import os
import pandas as pd
import json

# Build the full path
file_path =  r'../../scraped_data/yt_data.json'
try:
    data = pd.read_json(file_path)
except FileNotFoundError:
    print("File not found. Please check the path.")


In [48]:
from textblob import TextBlob

In [49]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   VideoID   5 non-null      object
 1   comments  5 non-null      object
 2   srt       5 non-null      object
dtypes: object(3)
memory usage: 248.0+ bytes


In [50]:
data.head()

,VideoID,comments,srt
0,yAvG9YHJMAs,"[[John 14:6 NKJV\r\n6 Jesus said to him, “I am...","-Welcome, everybody. Welcome, welcome, welcome..."
1,CdoOUAE71Ow,[[If Trump wins the election..you're finished ...,"no element found: line 1, column 0"
2,DV4TubkM-8U,"[[He'd be a better president than Trump !], [P...",keep walking this dramatic arrest of Ryan Ruth...
3,sSPzqcjE6Ms,[[All these economists in the comments warning...,"Andrew. Thank you. Well, former President Trum..."
4,uWpOxxhfG1Q,[[Is this the woman that claims that she's an ...,>>> DONALD TRUMP IS HAVING A >>> DONALD TRUMP ...


In [51]:
data.iloc[1]["comments"]

[["If Trump wins the election..you're finished as a democratic country!..Period!..shouldn't be even on the ballot for what he's doing and continues to do!"],
 ['Tous vacciné tous PROTEGE  les endormis !!!!???? Corompu  RASKIN!!! 👹'],
 ['Oh wow your not even good at talking smack anymore..good luck'],
 ['Decades ago, Republicans realized they were losing elections on policy.  Instead of revisiting their policies and revamping them and their GOP platform to appeal to a modern audience, a broader electorate, they came up with the "Culture Wars" instead and also made voting harder not easier.'],
 ['The Republicans are the ones cheating.'],
 ['IRONICALLY THE DEFINITION OF WHAT CONSTITUTES "THE STATE" IS THIS :- The superstructure of the State is the Judiciary and the Political - WHO CONTROLS BOTH OF THESE NOW - THE MAGA STOOGES!!MAGA CONTROLS THE STATE - THE DEEP STATE IS MAGA AND ITS ALREADY IN PLACE THATS WHY ITS SO DIFFICULT TO GET RID OF THEM.'],
 ['Read up on Hitler\'s "Night if the Lo

In [52]:
def analyze_sentiment(text):
    blob = TextBlob(text)
    # print(blob.sentiment.polarity)
    positive_sentiment = max(blob.sentiment.polarity, 0)
    positive_sentiment = round(positive_sentiment, 2)
    negative_sentiment = min(blob.sentiment.polarity, 0)
    negative_sentiment = round(negative_sentiment, 2)
    if positive_sentiment != 0:
        return "Positive",positive_sentiment
    return "Negative",negative_sentiment


In [53]:
import pandas as pd

def analyze_comments(row):
    positive_count = 0
    negative_count = 0
    
    for comment in row['comments']:
        sentiment, intensity = analyze_sentiment(comment[0])
        comment.append((sentiment, intensity))  # Assuming comment is a list
        
        if sentiment == 'Positive':
            positive_count += 1
        elif sentiment == 'Negative':
            negative_count += 1
    
    # Calculate percentages
    total_comments = positive_count + negative_count
    positive_percentage = (positive_count / total_comments) * 100 if total_comments > 0 else 0
    negative_percentage = (negative_count / total_comments) * 100 if total_comments > 0 else 0
    
    return pd.Series({
        "positive_comments": round(positive_percentage, 2),
        "negative_comments": round(negative_percentage, 2),
        "sentiment": analyze_sentiment(row["srt"])[0],
        "intensity": analyze_sentiment(row["srt"])[1]
    })

# Apply the function to each row and update the DataFrame
data[['positive_comments', 'negative_comments', 'sentiment', 'intensity']] = data.apply(analyze_comments, axis=1)


In [54]:
data.head()

,VideoID,comments,srt,positive_comments,negative_comments,sentiment,intensity
0,yAvG9YHJMAs,"[[John 14:6 NKJV\r\n6 Jesus said to him, “I am...","-Welcome, everybody. Welcome, welcome, welcome...",36.67,63.33,Positive,0.17
1,CdoOUAE71Ow,[[If Trump wins the election..you're finished ...,"no element found: line 1, column 0",30.00,70.00,Negative,0.00
2,DV4TubkM-8U,"[[He'd be a better president than Trump !, (Po...",keep walking this dramatic arrest of Ryan Ruth...,26.67,73.33,Negative,0.00
3,sSPzqcjE6Ms,[[All these economists in the comments warning...,"Andrew. Thank you. Well, former President Trum...",30.00,70.00,Positive,0.07
4,uWpOxxhfG1Q,[[Is this the woman that claims that she's an ...,>>> DONALD TRUMP IS HAVING A >>> DONALD TRUMP ...,23.33,76.67,Positive,0.19


In [55]:
data.iloc[0]["comments"]

[['John 14:6 NKJV\r\n6 Jesus said to him, “I am (A)the way, (B)the truth, and (C)the life. (D)No one comes to the Father (E)except through Me.\r\n\r\n*Did you know you’re part of the ultimate statistics, 10 out of 10 people die. One day your heart will stop beating and you will be dead! The Bible says you’ll then be judged by the holy and righteous God who created this universe.(Hebrew 9:27)You’ll be judged for every word, thought, and action in your life.(Matthew 12:36-37) Will you be innocent or guilty?\r\n\r\n*Let’s check the standard, the Ten Commandments. Have you ever lied (even a “white” lie)? (Revelation 21:8) Have you ever stolen anything (value is irrelevant and includes downloading or copying movies illegally)?  If you’ve lusted, you’ve committed adultery in your heart. (Matthew 5:27-30) (1 Corinthians 6:9-10)\r\n\r\nIf you’ve hated someone, the Bible says you’re a murderer(1 John 3:15). If you’ve done any of these things, you will be guilty, and end up in Hell. God is perfe

In [57]:

# Save to a JSON file
file_path = r"..\..\final_data\yt_data.json"

data.to_json(file_path, orient='records', indent=4)